In [6]:
import pandas as pd

df = pd.read_json('data/threads.json')
df.head()

,url,thread_name,init_post,comments
0,https://us.forums.blizzard.com/en/wow/t/player...,Players vs Developers,It hit me earlier today when I was wondering w...,[…pretty much.I don’t raid or dungeon so I hav...
1,https://us.forums.blizzard.com/en/wow/t/no-mot...,No Motivation for HoA Skills,The statement “it’s Blizzard game they can do ...,[More like the the devs hear us but don’t care...
2,https://us.forums.blizzard.com/en/wow/t/what-w...,What Would be the Most Jump-the-Shark Way to E...,Basically title.Allow us to use our whistle an...,[OMG YES! Though Blizz won’t do it because th...
3,https://us.forums.blizzard.com/en/wow/t/sugges...,Suggestion: Allow us to upgrade our Flight Mas...,Here is my submission.After the defeat of Azsh...,[We find out Sylvanas is pregnant with Anduin’...
4,https://us.forums.blizzard.com/en/wow/t/6-mont...,6 Months Was Long Enough,BFA released 7 months ago. I’ve experienced th...,[I agree on the time frame. I as well feel the...


In [2]:
import spacy
spacy.load('en_core_web_sm')

from spacy.lang.en import English
parser = English()


def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [5]:
import nltk

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [8]:
full_text = []

for index, row in df.iterrows():
    full_text.append(row['init_post'])
    full_text += row['comments']

In [11]:
nb_comments = len(full_text)
print("There are {} comments in the dataset".format(nb_comments))

There are 290121 comments in the dataset


In [19]:
processed_text = []

i=1
for comment in full_text:
    if(i%1000 ==0):
        print("Iteration {}".format(i))
    processed_text.append(prepare_text_for_lda(comment))
    i+=1

Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000
Iteration 10000
Iteration 11000
Iteration 12000
Iteration 13000
Iteration 14000
Iteration 15000
Iteration 16000
Iteration 17000
Iteration 18000
Iteration 19000
Iteration 20000
Iteration 21000
Iteration 22000
Iteration 23000
Iteration 24000
Iteration 25000
Iteration 26000
Iteration 27000
Iteration 28000
Iteration 29000
Iteration 30000
Iteration 31000
Iteration 32000
Iteration 33000
Iteration 34000
Iteration 35000
Iteration 36000
Iteration 37000
Iteration 38000
Iteration 39000
Iteration 40000
Iteration 41000
Iteration 42000
Iteration 43000
Iteration 44000
Iteration 45000
Iteration 46000
Iteration 47000
Iteration 48000
Iteration 49000
Iteration 50000
Iteration 51000
Iteration 52000
Iteration 53000
Iteration 54000
Iteration 55000
Iteration 56000
Iteration 57000
Iteration 58000
Iteration 59000
Iteration 60000
Iteration 61000
Iteration 62000
Iteration 63000
I

In [26]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_text)
corpus = [dictionary.doc2bow(text) for text in processed_text]

import pickle
pickle.dump(corpus, open('topic_modeling/corpus.pkl', 'wb'))
dictionary.save('topic_modeling/dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('topic_modeling/model5.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic[1])

In [36]:
import re

ldamodel.load('topic_modeling/model5.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(re.findall(r'"(.*?)"', topic[1]))

['reason', 'problem', 'break', 'agree', 'forget', 'absolutely', 'issue', 'disagree', 'love', 'leave']
['look', 'flying', 'account', 'cause', 'happy', 'report', 'minutes', 'remove', 'healing', 'green']
['people', 'player', 'would', 'group', 'guild', 'server', 'friend', 'think', 'really', 'blizzard']
['make', 'sense', 'happen', 'times', 'second', 'going', 'learn', 'expect', 'exactly', 'enough']
['level', 'character', 'expansion', 'content', 'legion', 'leveling', 'enjoy', 'quest', 'patch', 'grind']
['horde', 'alliance', 'sylvanas', 'faction', 'blood', 'kill', 'would', 'character', 'attack', 'saurfang']
['right', 'sound', 'super', 'stick', 'waiting', 'throw', 'maybe', 'starting', 'bunch', 'pandaren']
['elves', 'race', 'gnome', 'night', 'troll', 'ally', 'goblin', 'hunter', 'undead', 'tauren']
['think', 'forum', 'people', 'something', 'someone', 'point', 'would', 'wrong', 'saying', 'nothing']
['dungeon', 'raid', 'mythic', 'normal', 'heroic', 'piece', 'check', 'raider', 'casual', 'reply']
['r

In [31]:
ldamodel.show_topics(num_topics=20, num_words=10, log=False, formatted=True)

[(0,
  '0.045*"reason" + 0.043*"problem" + 0.042*"break" + 0.042*"agree" + 0.030*"forget" + 0.024*"absolutely" + 0.020*"issue" + 0.017*"disagree" + 0.016*"love" + 0.014*"leave"'),
 (1,
  '0.060*"look" + 0.060*"flying" + 0.039*"account" + 0.033*"cause" + 0.031*"happy" + 0.029*"report" + 0.027*"minutes" + 0.024*"remove" + 0.023*"healing" + 0.021*"green"'),
 (2,
  '0.077*"people" + 0.043*"player" + 0.025*"would" + 0.020*"group" + 0.015*"guild" + 0.015*"server" + 0.013*"friend" + 0.011*"think" + 0.010*"really" + 0.009*"blizzard"'),
 (3,
  '0.043*"make" + 0.039*"sense" + 0.035*"happen" + 0.031*"times" + 0.027*"second" + 0.022*"going" + 0.020*"learn" + 0.016*"expect" + 0.015*"exactly" + 0.015*"enough"'),
 (4,
  '0.065*"level" + 0.041*"character" + 0.040*"expansion" + 0.036*"content" + 0.021*"legion" + 0.018*"leveling" + 0.015*"enjoy" + 0.014*"quest" + 0.013*"patch" + 0.013*"grind"'),
 (5,
  '0.138*"horde" + 0.121*"alliance" + 0.046*"sylvanas" + 0.043*"faction" + 0.033*"blood" + 0.024*"kill" 

In [30]:
ldamodel.top_topics()

ValueError: One of texts or corpus has to be provided.